In [1]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import numpy as np
import seaborn as sbn
import pandas as pd
from datetime import datetime
import time
import gc
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from pandas.api.types import CategoricalDtype

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
plt.style.use('default') 
sbn.set(style="whitegrid") 

# **Satinización de archivos de edad y genero**

In [2]:
def postulantes_str2timestamp(s): # debo
    if type(s) is datetime:
        return s
    try:
        return datetime.strptime(s, '%Y-%m-%d')
    except:
        return None
def sanitize_postulante_genero_edad(df):
    df['sexo'] = df['sexo'].astype('category').fillna("NO_DECLARA")
    df = df.drop_duplicates(subset='idpostulante')
    df['fechanacimiento']= df['fechanacimiento'].map(postulantes_str2timestamp)
    mean_datetime = datetime.now()
    df['fechanacimiento']= df['fechanacimiento'].map(lambda it: it if it else mean_datetime)
    df['edad'] = df.fechanacimiento.apply(lambda it: 2018 - it.year)
    #df = df[df.edad >17][df.edad<105]
    return df.drop(['fechanacimiento'],1)
def generar_nivel_educativo(postulantes, educacion):
    #Considero como nivel educativo el maximo nivel que alcanzaron que fue completado.
    #Descarto estudios en curso o abandonados, quizas deba volver aca en algun momento
    tipo_estudio = CategoricalDtype(
        categories=["Otro","Secundario","Terciario/Técnico","Universitario","Posgrado","Master","Doctorado"],
        ordered=True
    )
    educacion = educacion[educacion.estado=='Graduado'].drop(['estado'],1)
    educacion.nombre= educacion.nombre.astype(tipo_estudio)
    educacion = educacion.sort_values(by='nombre',ascending=True)
    educacion = educacion.drop_duplicates(subset='idpostulante',keep='last')
    postulantes = postulantes.merge(educacion, on='idpostulante',how='left')
    postulantes = postulantes.rename(index=str,columns={'nombre':'nivel_educativo'})
    print("postulantes sin nivel educativo: ",postulantes.nivel_educativo.isna().value_counts())
    return postulantes
def satinizar_vistas(vistas):
    return vistas.drop_duplicates().rename(columns={'idAviso': 'idaviso'})

def sanitize_aviso_detalle(df):
    tipo_trabajo = CategoricalDtype(
        categories=["Full-time","Part-time","Teletrabajo","Por Horas","Pasantia","Temporario","Por Contrato","Fines de Semana","Primer empleo"],
        ordered=True #de 'mas fijo' a 'menos fijo'
    )
    df['tipo_de_trabajo'] = df['tipo_de_trabajo'].astype(tipo_trabajo)
    # el numero quizas amerite tratarse como hiperparametro
    #empresas_reconocidas = df.denominacion_empresa.value_counts().index[:100]
    #df.loc[:,'denominacion_empresa'] = df.denominacion_empresa.apply(lambda it: it if it in empresas_reconocidas else 'Otro')
    popularidad_empresa = CategoricalDtype(
        categories=df.denominacion_empresa.value_counts(ascending=True).index,
        ordered=True
    )
    df.loc[:,'denominacion_empresa'] = df.loc[:,'denominacion_empresa'].astype(popularidad_empresa)
    for columna in ['nombre_zona','nombre_area','nivel_laboral']:
        df.loc[:,columna] = df.loc[:,columna].astype('category')
    #df.nivel_laboral = df.nivel_laboral.fillna('Otro')
    #df.denominacion_empresa = df.denominacion_empresa.fillna('Otro')
    df = df.drop_duplicates(subset='idaviso')
    return df.drop(['ciudad','idpais','mapacalle'],1)
def sanitize_postulaciones(df):
    df['date'] = pd.to_datetime(df['fechapostulacion']).dt.date
    df['time'] = pd.to_datetime(df['fechapostulacion']).dt.time
    return df
def agregar_cantidad_anuncios(df, nombre_columna, anuncios):
    values = pd.DataFrame()
    value_counts = anuncios['idpostulante'].value_counts()
    values['idpostulante'] = value_counts.index
    values[nombre_columna] = value_counts.values
    df = pd.merge(df,values, on='idpostulante',how='left')
    df[nombre_columna] = df[nombre_columna].fillna(0)
    return df

In [3]:
#Antes que nada, me intriga si los archivos de 'datos_navent' y los de 'hasta15/4' tienen overlap, son redundantes o consecutivos
def leer_datos_entrenamiento():
    genero_edad_postulantes = pd.read_csv(
        'datos/datos_navent/fiuba_2_postulantes_genero_y_edad.csv'
    ).merge(
        pd.read_csv('datos/hasta_15_4/fiuba_2_postulantes_genero_y_edad.csv'),
        on=['idpostulante','fechanacimiento','sexo'],
        how='outer'
    )
    #veo que hay repetidos por id
    #veo que hay gente (4) con sexo '0'
    # Genero una funcion para satinizar archivos de postulantes_genero_edad
    genero_edad_postulantes = sanitize_postulante_genero_edad(genero_edad_postulantes)
    educacion_postulantes = pd.read_csv(
        'datos/datos_navent/fiuba_1_postulantes_educacion.csv'
    ).merge(pd.read_csv('datos/hasta_15_4/fiuba_1_postulantes_educacion.csv'))
    postulantes = generar_nivel_educativo(genero_edad_postulantes, educacion_postulantes)
    vistas = pd.concat([
        pd.read_csv('datos/hasta_15_4/fiuba_3_vistas.csv'),
        pd.read_csv('datos/datos_navent/fiuba_3_vistas.csv')
    ])
    vistas = satinizar_vistas(vistas)
    vistas.idaviso = vistas.idaviso.astype('int64')
    postulantes = agregar_cantidad_anuncios(postulantes, 'anuncios_vistos', vistas)
    postulaciones = pd.read_csv(
        'datos/datos_navent/fiuba_4_postulaciones.csv'
    ).merge(pd.read_csv('datos/hasta_15_4/fiuba_4_postulaciones.csv'))
    postulaciones = sanitize_postulaciones(postulaciones)
    avisos_detalle = pd.concat([
        pd.read_csv('datos/datos_navent/fiuba_6_avisos_detalle.csv'),
        pd.read_csv('datos/hasta_15_4/fiuba_6_avisos_detalle.csv'),
        pd.read_csv('datos/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')
    ])
    #ESTE DF TIENE MUCHOS DATOS NULOS EN LAS COLUMNAS DE 'CIUDAD' y 'MAPACALLE'. 
    #Decidimos eliminarlas ya que no nos parecieron muy relevantes para el analisis
    #idpais solo tiene valor 1, la descripcion nunca es nula
    #avisos_detalle.drop('descripcion',1,inplace=True)
    tipo_trabajo = CategoricalDtype(
        categories=["Full-time","Part-time","Teletrabajo","Por Horas","Pasantia","Temporario","Por Contrato","Fines de Semana","Primer empleo"],
        ordered=True #de 'mas fijo' a 'menos fijo'
    )
    avisos_detalle.nombre_area.value_counts()# hay muchas areas, no van a tener su propia categoria
    avisos_detalle = sanitize_aviso_detalle(avisos_detalle)
    return postulantes, avisos_detalle, vistas, postulaciones

In [31]:
def leer_datos_predicciones():
    vistas = satinizar_vistas(pd.read_csv('datos/desde_15_4/fiuba_3_vistas.csv'))
    avisos = sanitize_aviso_detalle(pd.concat([
        pd.read_csv('datos/desde_15_4/fiuba_6_avisos_detalle.csv'),
        pd.read_csv('datos/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')#
    ]))
    genero_edad_postulantes = sanitize_postulante_genero_edad(
        pd.concat([
            pd.read_csv('datos/desde_15_4/fiuba_2_postulantes_genero_y_edad.csv'),
            pd.read_csv('datos/hasta_15_4/fiuba_2_postulantes_genero_y_edad.csv')
        ])
    )
    postulantes = generar_nivel_educativo(genero_edad_postulantes, pd.read_csv('datos/desde_15_4/fiuba_1_postulantes_educacion.csv'))
    postulantes = agregar_cantidad_anuncios(postulantes, 'anuncios_vistos', vistas)
    template_submit = pd.read_csv('datos/template_resultado.csv')
    return postulantes, avisos, vistas, template_submit

In [35]:
postulantes, avisos_detalle, vistas, postulaciones = leer_datos_entrenamiento()
gc.collect()

/home/vasectomio/.virtualenv/datos/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/vasectomio/.virtualenv/datos/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/vasectomio/.virtualenv/datos/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

postulantes sin nivel educativo:  True     338217
False     69929
Name: nivel_educativo, dtype: int64


234

# Pruebo un random forest, con solamente las variables numericas/categoricas

In [6]:
def codificar_categoricas(df):
    categorical_coulumns = df.select_dtypes(['category']).columns
    df[categorical_coulumns] = df[categorical_coulumns].apply(lambda it: it.cat.codes)
    return df
# Merger puede ser un dataframe de vistas (caso entrenamiento) o de los casos a predecir(caso prediccion/armar submit)
def armar_dataframe_forest(postulantes, avisos, merger, postulaciones, proporcion_entrenamiento):
    avisos_forest =  avisos.drop(['titulo','descripcion'],1)
    joined = postulantes.merge(
        merger.loc[:,['idpostulante','idaviso']],on='idpostulante',how='left'
    ).merge(
        avisos_forest,on='idaviso',how='left'
    )
    if proporcion_entrenamiento != 0 and postulaciones is not None:
        postulaciones_forest = postulaciones.loc[:,['idaviso','idpostulante']]
        postulaciones_forest['sepostulo'] = 1
        joined = joined.merge(
            postulaciones_forest,on=['idaviso','idpostulante'],how='left'
        )
        joined.sepostulo = joined.sepostulo.fillna(0)
        joined['es_entrenamiento'] = np.random.uniform(0, 1, joined.shape[0]) <= proporcion_entrenamiento
    features = [it for it in joined.columns if it not in ['id','idaviso','idpostulante','es_entrenamiento','sepostulo']]
    joined = codificar_categoricas(joined)
    return joined, features

In [36]:
df_forest, features = armar_dataframe_forest(postulantes, avisos_detalle, vistas, postulaciones, 0.90)
classifier = RandomForestClassifier(n_jobs=5, random_state=0)
#classifier.fit(df_entrenamiento[features], df_entrenamiento.sepostulo.astype('int64').values)
train, test = df_forest[df_forest.es_entrenamiento],df_forest[~df_forest.es_entrenamiento]
classifier.fit(train[features], train.sepostulo.astype('int64').values)

results = classifier.predict(test[features])
roc_auc_score(y_true=test.sepostulo.astype('int64').values,y_score=results)

0.8391077208775085

## A partir de aqui, tengo armado el dataframe de entrenamiento
por lo que no necesito ya los dataframes 'crudos'

In [37]:
postulantes, avisos, vistas, submit = leer_datos_predicciones()
del postulaciones
gc.collect()

/home/vasectomio/.virtualenv/datos/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/vasectomio/.virtualenv/datos/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/vasectomio/.virtualenv/datos/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

postulantes sin nivel educativo:  True     216660
False    203830
Name: nivel_educativo, dtype: int64


228

In [34]:
df_predicciones = submit.merge(
    avisos, on='idaviso',how='inner'
).merge(
    postulantes, on='idpostulante', how='inner'
)
df_predicciones = codificar_categoricas(df_predicciones)
features = [it for it in df_predicciones.columns if it not in ['idaviso','idpostulante','es_entrenamiento','sepostulo']]
df_predicciones.shape

(99988, 14)

In [38]:
def clasificar_random_forest(classifier, postulantes, avisos, template_submit):
    df_predicciones = template_submit.merge(
        postulantes, on='idpostulante', how='inner'
    ).merge(
        avisos, on='idaviso',how='inner'
    )
    df_predicciones = codificar_categoricas(df_predicciones)
    df_predicciones = df_predicciones.drop(['titulo','descripcion'],1)
    features = [it for it in df_predicciones.columns if it not in ['id','idaviso','idpostulante','es_entrenamiento','sepostulo']]
    df_predicciones['sepostulo'] = classifier.predict(df_predicciones[features])
    return df_predicciones
tentativo_submit = clasificar_random_forest(classifier, postulantes, avisos, submit)
tentativo_submit.isna().any()

id                      False
idaviso                 False
idpostulante            False
sexo                    False
edad                    False
nivel_educativo         False
anuncios_vistos         False
nombre_zona             False
tipo_de_trabajo         False
nivel_laboral           False
nombre_area             False
denominacion_empresa    False
sepostulo               False
dtype: bool

In [ ]:
#postulantes.drop(['fechanacimiento'],1).merge(
    #vistas.loc[:,['idpostulante','idaviso']],on='idpostulante',how='left'
#).merge(

# No hay vistas que no tengan un 
nuevas_vistas.loc[:,['idpostulante','idaviso']].merge(nuevos_avisos, on='idaviso',how='inner').isna().shape#.any()

In [ ]:
classifier = RandomForestClassifier(n_jobs=5, random_state=0)


In [ ]:
param_grid = dict(max_features = [0.5,0.75],
                    min_samples_leaf= [2,4,7,12,19],
                    n_estimators = [5,10,30,50]
                 )
# caso entrenado con muchos datos
df_forest, features = armar_dataframe_forest(postulantes, avisos_detalle, vistas, postulaciones, 0.75)
train, test = df_forest[df_forest.es_entrenamiento],df_forest[~df_forest.es_entrenamiento]
classifier = RandomForestClassifier(n_jobs=5, random_state=0)
grid_bien_entrenada = GridSearchCV(estimator=classifier, n_jobs=2, param_grid=param_grid)
grid_bien_entrenada.fit(train[features], train.sepostulo.astype('int64').values)
print("La matriz bien entrenada logro un puntaje de ", grid_bien_entrenada.best_score_," con parametros ", grid_bien_entrenada.best_params_)
#Caso entrenado con pocos datos
df_forest, features = armar_dataframe_forest(postulantes, avisos_detalle, vistas, postulaciones, 0.25)
train, test = df_forest[df_forest.es_entrenamiento],df_forest[~df_forest.es_entrenamiento]
classifier = RandomForestClassifier(n_jobs=5, random_state=0)
grid_poco_entrenada = GridSearchCV(estimator=classifier, n_jobs=2, param_grid=param_grid)
grid_poco_entrenada.fit(train[features], train.sepostulo.astype('int64').values)
print("La matriz poco entrenada logro un puntaje de ", grid_poco_entrenada.best_score_," con parametros ", grid_poco_entrenada.best_params_)

# Generar un resultado

In [ ]:
tentativa = pd.read_csv('datos/template_resultado.csv')
submit = pd.DataFrame()
submit['id'] = tentativa['id']
submit = submit.set_index('id')
submit['sepostulo'] =0
submit.to_csv(path_or_buf='submit.csv')
submit

# Defino los shingles de cada anuncio

In [ ]:
def shingles(string, n = 3):
    return [string[i:i + n] for i in range(len(string) - n + 1)]

def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    print(set(list1).intersection(list2))
    union = (len(list1) + len(list2)) - intersection
    print(union)
    return float(intersection / union)
#avisos_detalle['shingles_descripcion'] = avisos_detalle.descripcion.apply(shingles)
#avisos_detalle['shingles_titulo'] = avisos_detalle.titulo.apply(shingles)